In [1]:
import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# sitemap

In [2]:
sections = ['US', 'International', 'Politics']
offsets = range(0, 10000-10, 10)

products = itertools.product(sections,offsets)

df= pd.DataFrame([ [section, offset] for (section,offset) in products],columns=['section', 'offset'])
df1 = df.copy()

In [3]:
df.shape

(2997, 2)

In [4]:
df['page'] = \
'https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section='+df['section']+\
    '&totalrecords=true&offset='+ df['offset'].astype(str)
df1['page'] = \
'https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date&type=Story&section='+df['section']+\
    '&totalrecords=true&offset='+ df1['offset'].astype(str)

df= pd.concat([df, df1]).reset_index(drop=True)
df

,section,offset,page
0,US,0,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section=US&totalrecords=true&offset=0
1,US,10,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section=US&totalrecords=true&offset=10
2,US,20,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section=US&totalrecords=true&offset=20
3,US,30,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section=US&totalrecords=true&offset=30
4,US,40,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date-asc&type=Story&section=US&totalrecords=true&offset=40
...,...,...,...
5989,Politics,9940,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date&type=Story&section=Politics&totalrecords=true&offset=9940
5990,Politics,9950,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date&type=Story&section=Politics&totalrecords=true&offset=9950
5991,Politics,9960,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date&type=Story&section=Politics&totalrecords=true&offset=9960
5992,Politics,9970,https://abcnews.go.com/meta/api/search?q=news&limit=10&sort=date&type=Story&section=Politics&totalrecords=true&offset=9970


In [5]:
def query_json(page):
    try:
        data = requests.get(page).json()
        rows = data['item']
        return rows
    except:
        return None

df['rows']= None

pandarallel.initialize(nb_workers =64, progress_bar=True)
df['rows']= df['page'].parallel_apply(query_json)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
df[df.rows.isnull()]

,section,offset,page,rows


In [9]:
df= df.explode('rows').reset_index(drop=True)
df = pd.json_normalize(df.rows)
df = df[['title', 'link', 'pubDate', 'description', 'guid.text',  'category.text']]
df.columns =['title', 'url', 'date', 'abstract', 'id',  'category']
df

,title,url,date,abstract,id,category
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,"Fri, 29 Dec 2000 18:29:02 +0000","100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,"Mon, 05 Mar 2001 05:00:00 +0000","2 Dead, 13 Wounded in Calif. High School",news-92482,us
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,"Wed, 07 Mar 2001 05:00:00 +0000",Suspect Is In Custody,news-93896,us
3,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,"Thu, 16 Aug 2001 21:03:33 +0000",Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us
4,"Delta Flight Detained, FBI Investigates",https://abcnews.go.com/US/delta-flight-detained-fbi-investigates/story?id=92331,"Sun, 14 Oct 2001 02:06:07 +0000",Four Men Purchsed One Way Tickets,news-92331,us
...,...,...,...,...,...,...
59935,Melania Trump returns home to White House after hospitalization,https://abcnews.go.com/Politics/melania-trump-returns-home-white-house-hospitalization/story?id=55293210,"Sat, 19 May 2018 18:46:00 +0000","""She is resting comfortably and remains in high spirits,"" her office said in a statement.",news-55293210,politics
59936,Lawyer for Trump's fixer seeks to block Avenatti from participating in Cohen case,https://abcnews.go.com/Politics/lawyer-trumps-fixer-seeks-block-avenatti-participating-cohen/story?id=55294617,"Sat, 19 May 2018 15:51:00 +0000",The war of words between Michael Cohen and Michael Avenatti is heating up again.,news-55294617,politics
59937,Trump fans flames about alleged 'spy' within campaign while top lawyer tamps them down,https://abcnews.go.com/Politics/trump-fans-flames-alleged-spy-campaign-top-lawyer/story?id=55261255,"Fri, 18 May 2018 22:16:00 +0000","President Donald Trump is amplifying conservative voices who are alleging, unsubstantiated, that the Department of Justice planted a spy inside his presidential campaign.",news-55261255,politics
59938,'We are going to take back the country we love': Hillary Clinton,https://abcnews.go.com/Politics/back-country-love-hillary-clinton/story?id=55267784,"Fri, 18 May 2018 19:53:50 +0000",Clinton urges Democratic women to return to basics and stress values.,news-55267784,politics


In [11]:
df = df[~df.duplicated()].reset_index(drop=True)

In [13]:
df.date = pd.to_datetime(df.date)
df = df.sort_values('date').reset_index(drop=True)
df

,title,url,date,abstract,id,category
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us
...,...,...,...,...,...,...
50229,Police officer died of 'natural' causes day after Capitol riot: Medical examiner,https://abcnews.go.com/Politics/police-officer-died-natural-day-capitol-riot-medical/story?id=77173111,2021-04-19 22:55:16+00:00,"U.S. Capitol Police Officer Brian Sicknick died of ""natural"" causes one day after a violent mob stormed the U.S. Capitol, the D.C. medical examiner announced on Monday.",news-77173111,politics
50230,Chauvin defense asks for mistrial based on Rep. Maxine Waters' 'guilty' comments,https://abcnews.go.com/Politics/chauvin-defense-asks-mistrial-based-rep-maxine-waters/story?id=77164878,2021-04-19 23:02:00+00:00,"House Republican Leader Kevin McCarthy accused Democratic Rep. Maxine Waters of ""inciting violence"" for her weekend comments about racial justice protests.",news-77164878,politics
50231,"Biden will speak after Chauvin verdict, has been in touch with local authorities: White House",https://abcnews.go.com/Politics/biden-speak-chauvin-verdict-touch-local-authorities-white/story?id=77172231,2021-04-19 23:55:00+00:00,"According to the White House, President Joe Biden is set to speak after the verdict comes out of the Derek Chauvin trial in Minneapolis.",news-77172231,politics
50232,How some Republican lawmakers are fighting conservatives' COVID-19 vaccine hesitancy,https://abcnews.go.com/Politics/republican-lawmakers-fighting-conservatives-covid-19-vaccine-hesitancy/story?id=77169403,2021-04-20 00:32:28+00:00,Republicans' hesitancy towards the COVID-19 vaccine could impact when the country reaches herd immunity.,news-77169403,politics


In [16]:
df.to_csv('abcnews_2000_2021_raw.csv', index=False)

# Filtering


In [37]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [39]:
from collections import Counter
pandarallel.initialize()

head = df['title'] + ' ' + df['url']+ ' ' +df['abstract'] + ' ' + df['category']
df['head_in'] = head.parallel_apply(find_relevant)
df['head_out'] = head.parallel_apply(find_exclude)
df['head_in_count']= df['head_in'].str.len()
df['head_out_count']= df['head_out'].str.len()
df

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0
...,...,...,...,...,...,...,...,...,...,...
50229,Police officer died of 'natural' causes day after Capitol riot: Medical examiner,https://abcnews.go.com/Politics/police-officer-died-natural-day-capitol-riot-medical/story?id=77173111,2021-04-19 22:55:16+00:00,"U.S. Capitol Police Officer Brian Sicknick died of ""natural"" causes one day after a violent mob stormed the U.S. Capitol, the D.C. medical examiner announced on Monday.",news-77173111,politics,"{'police': 3, 'officer': 3, 'riot': 2, 'politics': 2, 'violen': 1}",{},5,0
50230,Chauvin defense asks for mistrial based on Rep. Maxine Waters' 'guilty' comments,https://abcnews.go.com/Politics/chauvin-defense-asks-mistrial-based-rep-maxine-waters/story?id=77164878,2021-04-19 23:02:00+00:00,"House Republican Leader Kevin McCarthy accused Democratic Rep. Maxine Waters of ""inciting violence"" for her weekend comments about racial justice protests.",news-77164878,politics,"{'politics': 2, 'republican': 1, 'democratic': 1, 'violen': 1, 'justice': 1, 'protest': 1}",{},6,0
50231,"Biden will speak after Chauvin verdict, has been in touch with local authorities: White House",https://abcnews.go.com/Politics/biden-speak-chauvin-verdict-touch-local-authorities-white/story?id=77172231,2021-04-19 23:55:00+00:00,"According to the White House, President Joe Biden is set to speak after the verdict comes out of the Derek Chauvin trial in Minneapolis.",news-77172231,politics,"{'authorities': 2, 'politics': 2, 'president': 1}",{},3,0
50232,How some Republican lawmakers are fighting conservatives' COVID-19 vaccine hesitancy,https://abcnews.go.com/Politics/republican-lawmakers-fighting-conservatives-covid-19-vaccine-hesitancy/story?id=77169403,2021-04-20 00:32:28+00:00,Republicans' hesitancy towards the COVID-19 vaccine could impact when the country reaches herd immunity.,news-77169403,politics,"{'republican': 3, 'conservative': 2, 'politics': 2}",{},3,0


In [40]:
df.to_csv('abcnews_1.csv', index=False)

# threshold

In [2]:
df = pd.read_csv('abcnews_1.csv', header=0)
df

,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0
...,...,...,...,...,...,...,...,...,...,...
50229,Police officer died of 'natural' causes day after Capitol riot: Medical examiner,https://abcnews.go.com/Politics/police-officer-died-natural-day-capitol-riot-medical/story?id=77173111,2021-04-19 22:55:16+00:00,"U.S. Capitol Police Officer Brian Sicknick died of ""natural"" causes one day after a violent mob stormed the U.S. Capitol, the D.C. medical examiner announced on Monday.",news-77173111,politics,"{'police': 3, 'officer': 3, 'riot': 2, 'politics': 2, 'violen': 1}",{},5,0
50230,Chauvin defense asks for mistrial based on Rep. Maxine Waters' 'guilty' comments,https://abcnews.go.com/Politics/chauvin-defense-asks-mistrial-based-rep-maxine-waters/story?id=77164878,2021-04-19 23:02:00+00:00,"House Republican Leader Kevin McCarthy accused Democratic Rep. Maxine Waters of ""inciting violence"" for her weekend comments about racial justice protests.",news-77164878,politics,"{'politics': 2, 'republican': 1, 'democratic': 1, 'violen': 1, 'justice': 1, 'protest': 1}",{},6,0
50231,"Biden will speak after Chauvin verdict, has been in touch with local authorities: White House",https://abcnews.go.com/Politics/biden-speak-chauvin-verdict-touch-local-authorities-white/story?id=77172231,2021-04-19 23:55:00+00:00,"According to the White House, President Joe Biden is set to speak after the verdict comes out of the Derek Chauvin trial in Minneapolis.",news-77172231,politics,"{'authorities': 2, 'politics': 2, 'president': 1}",{},3,0
50232,How some Republican lawmakers are fighting conservatives' COVID-19 vaccine hesitancy,https://abcnews.go.com/Politics/republican-lawmakers-fighting-conservatives-covid-19-vaccine-hesitancy/story?id=77169403,2021-04-20 00:32:28+00:00,Republicans' hesitancy towards the COVID-19 vaccine could impact when the country reaches herd immunity.,news-77169403,politics,"{'republican': 3, 'conservative': 2, 'politics': 2}",{},3,0


In [3]:
df.head_in_count.describe(), df.head_out_count.describe()

(count    50234.000000
 mean         2.045129
 std          1.726546
 min          0.000000
 25%          1.000000
 50%          2.000000
 75%          3.000000
 max         15.000000
 Name: head_in_count, dtype: float64,
 count    50234.000000
 mean         0.306426
 std          0.623021
 min          0.000000
 25%          0.000000
 50%          0.000000
 75%          0.000000
 max          8.000000
 Name: head_out_count, dtype: float64)

In [4]:
df1 = df.copy()
print(df1.shape[0])

head_in = df['head_in_count']
head_out = df['head_out_count']

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df1[remove_index== True].index

df1 = df1[~df1.index.isin(remove_index)].reset_index(drop=True)
df1

50234


,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0
...,...,...,...,...,...,...,...,...,...,...
45700,Police officer died of 'natural' causes day after Capitol riot: Medical examiner,https://abcnews.go.com/Politics/police-officer-died-natural-day-capitol-riot-medical/story?id=77173111,2021-04-19 22:55:16+00:00,"U.S. Capitol Police Officer Brian Sicknick died of ""natural"" causes one day after a violent mob stormed the U.S. Capitol, the D.C. medical examiner announced on Monday.",news-77173111,politics,"{'police': 3, 'officer': 3, 'riot': 2, 'politics': 2, 'violen': 1}",{},5,0
45701,Chauvin defense asks for mistrial based on Rep. Maxine Waters' 'guilty' comments,https://abcnews.go.com/Politics/chauvin-defense-asks-mistrial-based-rep-maxine-waters/story?id=77164878,2021-04-19 23:02:00+00:00,"House Republican Leader Kevin McCarthy accused Democratic Rep. Maxine Waters of ""inciting violence"" for her weekend comments about racial justice protests.",news-77164878,politics,"{'politics': 2, 'republican': 1, 'democratic': 1, 'violen': 1, 'justice': 1, 'protest': 1}",{},6,0
45702,"Biden will speak after Chauvin verdict, has been in touch with local authorities: White House",https://abcnews.go.com/Politics/biden-speak-chauvin-verdict-touch-local-authorities-white/story?id=77172231,2021-04-19 23:55:00+00:00,"According to the White House, President Joe Biden is set to speak after the verdict comes out of the Derek Chauvin trial in Minneapolis.",news-77172231,politics,"{'authorities': 2, 'politics': 2, 'president': 1}",{},3,0
45703,How some Republican lawmakers are fighting conservatives' COVID-19 vaccine hesitancy,https://abcnews.go.com/Politics/republican-lawmakers-fighting-conservatives-covid-19-vaccine-hesitancy/story?id=77169403,2021-04-20 00:32:28+00:00,Republicans' hesitancy towards the COVID-19 vaccine could impact when the country reaches herd immunity.,news-77169403,politics,"{'republican': 3, 'conservative': 2, 'politics': 2}",{},3,0


# Crawling

In [5]:
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        article.download()
        article.parse()
        return article.text
    except:
        print('error %s'%url)
        return

In [51]:
# df1.to_csv('abcnews_2.csv', index=False)

In [ ]:
# df = pd.read_csv('abcnews_2.csv',header=0)

In [6]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size = 16*5
i=0
while i < df1.shape[0]:
    print(i, end=',')
    df1.loc[i: i+ batch_size-1,'text']=\
    df1.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_story).values
    i += batch_size

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,80,160,240,320,400,480,560,640,720,800,880,960,1040,1120,1200,1280,1360,1440,1520,1600,1680,1760,1840,1920,2000,2080,2160,2240,2320,2400,2480,2560,2640,2720,2800,2880,2960,3040,3120,3200,3280,3360,3440,3520,3600,3680,3760,3840,3920,4000,4080,4160,4240,4320,4400,4480,4560,4640,4720,4800,4880,4960,5040,5120,5200,5280,5360,5440,5520,5600,5680,5760,5840,5920,6000,6080,6160,error https://abcnews.go.com/International/amanda-knox-family-sees-glimmers-hope/story?id=14083153
error https://abcnews.go.com/Politics/osama-bin-laden-raid-bitter-us-split-pakistan/story?id=13561191
error https://abcnews.go.com/US/texas-teen-2000-found-parking-lot/story?id=13588776
error https://abcnews.go.com/US/memphis-tennessee-flooding-residents-brace-rising-mississippi-river/story?id=13559041
6240,error https://abcnews.go.com/US/dominique-strauss-kahn-sex-case-alleged-vic

In [7]:
df1.to_csv('abcnews_2.csv', index=False)

In [11]:
pandarallel.initialize(nb_workers=2, progress_bar=True)
df1.loc[df1.text.isnull(), 'text'] =\
df1.loc[df1.text.isnull(), 'url'].parallel_apply(extract_story).values

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:
# df= df[~df.text.isnull()]
df1.text.str.len().describe()

count     45705.000000
mean       4667.764730
std        5760.909265
min          74.000000
25%        2278.000000
50%        3500.000000
75%        5336.000000
max      100000.000000
Name: text, dtype: float64

In [16]:
df1 = df1[df1.text.str.len()>200].reset_index(drop=True)
df1

,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count,text
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0,"Dec. 31, 2000  -- For New Year’s Eve 1900, New Yorkers pulled out all the stops to welcome the new century with fireworks, a lineup of bands and speakers, and blazing displays of electric lights. It all followed a mountain of hype in the media about the coming of the 20th century and the passing of the 19th.\n\nIt’s 100 years later. A few cities are planning big celebrations.\n\nBy and large, though, we’ve been there, done that.\n\nLike us, our 1800 and 1900 ancestors aired the debate on the start of the new century. And like us, scientists and scholars insisted upon the conclusion that t..."
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0,"March 5, 2001 -- Smiling as he pulled the trigger, a 15-year-old freshman shot and killed two students and wounded 13 other people at a San Diego-area high school this morning, authorities and witnesses said.\n\nSheriff's deputies took the suspect into custody at Santana High School in Santee, just east of San Diego, where the deadly rampage took place around 9:20 a.m.\n\nTwo students were killed and 13 other people wounded, said San Diego County Sheriff William Kolender.\n\nA 14-year-old boy died at the scene, and a 15-year-old boy who had been shot in the head died at Grossmont Hospital...."
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0," -- An eighth-grade girl is in custody afer shooting a fellow female student today during lunch at a Roman Catholic high school in Williamsport, Pa., authorities said.\n\nWilliamsport Police said an argument between the two girls sparked the shooting, which occurred at around noon at Bishop Neumann Junior-Senior High. They said the suspect, who is 14, shot the 13-year-old victim from behind.\n\nPolice said the girls knew each other and that the shooting was ""not an act of random violence."" The girls had been engaged in an ongoing feud, according to Lycoming County District Attorney TomMar..."
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0,"March 28, 2001 -- The mystery surrounding a headless body found in the woods outside Kiev has created a dark cloud over the Ukrainian head of state.\n\nPresident Leonid Kuchma has been caught up in the storm around the body, believed to be that of opposition journalist Heorhiy Gongadze, ever since a former presidential security officer now in hiding in Europe announced he had tape recordings of the president talking about getting rid of the reporter.\n\nBut more than four months after the body was found, it has not been conclusively identified, and the integrity of the tapes has been quest..."
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0,"Aug. 16, 2001 -- Irvin Rosenfeld needs marijuana.\n\nHe's not an addict, but the 49-year-old stockbroker has a rare and painful bone disease that 19 years ago led the federal government to not only prescribe marijuana for him, but to provide him with the narcotic as well.\n\nHe is one of only eight people in the United States legally allowed to use marijua

In [20]:
df1 = df1[~df1.text.duplicated()]

In [21]:
df1.to_csv('abcnews_3.csv', index=False)

# Merge

In [2]:
df = pd.read_csv('abcnews_3.csv', header=0)
df

,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count,text
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0,"Dec. 31, 2000  -- For New Year’s Eve 1900, New Yorkers pulled out all the stops to welcome the new century with fireworks, a lineup of bands and speakers, and blazing displays of electric lights. It all followed a mountain of hype in the media about the coming of the 20th century and the passing of the 19th.\n\nIt’s 100 years later. A few cities are planning big celebrations.\n\nBy and large, though, we’ve been there, done that.\n\nLike us, our 1800 and 1900 ancestors aired the debate on the start of the new century. And like us, scientists and scholars insisted upon the conclusion that t..."
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0,"March 5, 2001 -- Smiling as he pulled the trigger, a 15-year-old freshman shot and killed two students and wounded 13 other people at a San Diego-area high school this morning, authorities and witnesses said.\n\nSheriff's deputies took the suspect into custody at Santana High School in Santee, just east of San Diego, where the deadly rampage took place around 9:20 a.m.\n\nTwo students were killed and 13 other people wounded, said San Diego County Sheriff William Kolender.\n\nA 14-year-old boy died at the scene, and a 15-year-old boy who had been shot in the head died at Grossmont Hospital...."
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0," -- An eighth-grade girl is in custody afer shooting a fellow female student today during lunch at a Roman Catholic high school in Williamsport, Pa., authorities said.\n\nWilliamsport Police said an argument between the two girls sparked the shooting, which occurred at around noon at Bishop Neumann Junior-Senior High. They said the suspect, who is 14, shot the 13-year-old victim from behind.\n\nPolice said the girls knew each other and that the shooting was ""not an act of random violence."" The girls had been engaged in an ongoing feud, according to Lycoming County District Attorney TomMar..."
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0,"March 28, 2001 -- The mystery surrounding a headless body found in the woods outside Kiev has created a dark cloud over the Ukrainian head of state.\n\nPresident Leonid Kuchma has been caught up in the storm around the body, believed to be that of opposition journalist Heorhiy Gongadze, ever since a former presidential security officer now in hiding in Europe announced he had tape recordings of the president talking about getting rid of the reporter.\n\nBut more than four months after the body was found, it has not been conclusively identified, and the integrity of the tapes has been quest..."
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0,"Aug. 16, 2001 -- Irvin Rosenfeld needs marijuana.\n\nHe's not an addict, but the 49-year-old stockbroker has a rare and painful bone disease that 19 years ago led the federal government to not only prescribe marijuana for him, but to provide him with the narcotic as well.\n\nHe is one of only eight people in the United States legally allowed to use marijua

In [3]:
df.sort_values('date')

,title,url,date,abstract,id,category,head_in,head_out,head_in_count,head_out_count,text
0,1900's New Century Hype Was Millennial,https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978,2000-12-29 18:29:02+00:00,"100 Years Ago, Our Ancestors Waited Until 1901 to Celebrate",news-89978,us,{},{},0,0,"Dec. 31, 2000  -- For New Year’s Eve 1900, New Yorkers pulled out all the stops to welcome the new century with fireworks, a lineup of bands and speakers, and blazing displays of electric lights. It all followed a mountain of hype in the media about the coming of the 20th century and the passing of the 19th.\n\nIt’s 100 years later. A few cities are planning big celebrations.\n\nBy and large, though, we’ve been there, done that.\n\nLike us, our 1800 and 1900 ancestors aired the debate on the start of the new century. And like us, scientists and scholars insisted upon the conclusion that t..."
1,"2 Dead, 13 Hurt in Calif. School Shooting",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482,2001-03-05 05:00:00+00:00,"2 Dead, 13 Wounded in Calif. High School",news-92482,us,"{'dead': 3, 'shooting': 2}",{},2,0,"March 5, 2001 -- Smiling as he pulled the trigger, a 15-year-old freshman shot and killed two students and wounded 13 other people at a San Diego-area high school this morning, authorities and witnesses said.\n\nSheriff's deputies took the suspect into custody at Santana High School in Santee, just east of San Diego, where the deadly rampage took place around 9:20 a.m.\n\nTwo students were killed and 13 other people wounded, said San Diego County Sheriff William Kolender.\n\nA 14-year-old boy died at the scene, and a 15-year-old boy who had been shot in the head died at Grossmont Hospital...."
2,Shot Fired At Pa. High School,https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896,2001-03-07 05:00:00+00:00,Suspect Is In Custody,news-93896,us,"{'shot': 2, 'suspect': 1}",{},2,0," -- An eighth-grade girl is in custody afer shooting a fellow female student today during lunch at a Roman Catholic high school in Williamsport, Pa., authorities said.\n\nWilliamsport Police said an argument between the two girls sparked the shooting, which occurred at around noon at Bishop Neumann Junior-Senior High. They said the suspect, who is 14, shot the 13-year-old victim from behind.\n\nPolice said the girls knew each other and that the shooting was ""not an act of random violence."" The girls had been engaged in an ongoing feud, according to Lycoming County District Attorney TomMar..."
3,"FBI, Private Sleuths Probe Ukraine Mystery",https://abcnews.go.com/International/story?id=81346&page=1,2001-03-27 17:51:45+00:00,"FBI, Private Detectives Called in to Help Solve Ukraine Murder Mystery",news-81346,international,{'murder': 1},{},1,0,"March 28, 2001 -- The mystery surrounding a headless body found in the woods outside Kiev has created a dark cloud over the Ukrainian head of state.\n\nPresident Leonid Kuchma has been caught up in the storm around the body, believed to be that of opposition journalist Heorhiy Gongadze, ever since a former presidential security officer now in hiding in Europe announced he had tape recordings of the president talking about getting rid of the reporter.\n\nBut more than four months after the body was found, it has not been conclusively identified, and the integrity of the tapes has been quest..."
4,Pot Smoker Demands Apology From Delta,https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638,2001-08-16 21:03:33+00:00,Legal Marijuana User Wants Apology for Ouster From Plane,news-92638,us,{'legal': 1},{},1,0,"Aug. 16, 2001 -- Irvin Rosenfeld needs marijuana.\n\nHe's not an addict, but the 49-year-old stockbroker has a rare and painful bone disease that 19 years ago led the federal government to not only prescribe marijuana for him, but to provide him with the narcotic as well.\n\nHe is one of only eight people in the United States legally allowed to use marijua

In [5]:
df_others = pd.read_csv('abc_13_21.csv', header=0)
df_others

,url,date,title,authors,text
0,http://abcnews.go.com/Politics/expect-donald-trumps-major-immigration-speech/story?id=41717367,20160829,What to Expect From Donald Trump's 'Major' Immigration Speech,"['Abc News', 'Meghan Keneally', 'August']"," -- Donald Trump has been talking about his immigration plans for weeks and now he's teasing a ""major speech"" he'll be giving on the topic this week.\n\nNot much is known about his plans for the immigration speech besides what he posted on Twitter last night.\n\nThe speech comes after Trump and his campaign manager Kellyanne Conway gave differing statements about his plans during several appearances last week.\n\nConway stepped further back from Trump's earlier suggestions of a deportation force during a television appearance this weekend, saying that he ""is not talking about a deportatio..."
1,https://web.archive.org/web/20151130180232/http://abcnews.go.com/US/wireStory/boeing-17-leaves-southern-california-assembly-plant-35478959,20151130,Last Boeing C-17 Leaves Southern California Assembly Plant,['Abc News'],"The last C-17 Globemaster III built at a Southern California Boeing plant soared into history on Sunday with a flyover that marked the end of an era for the region's once-thriving aerospace industry.\n\nThe enormous cargo jet was cheered in Long Beach as if roared over the heads of an estimated 1,000 onlookers, many of them Boeing employees, spokesman Felix Sanchez said.\n\nThe jet, which can hold more than 80 tons of cargo, will be housed in San Antonio, Texas, until it is delivered to the Qatar Emiri Air Force early next year.\n\n""This is truly the end of an era. It's a sad day, but one ..."
2,http://abcnews.go.com/US/witness-defends-officer-rammed-suspect-patrol-car/story?id=30358441,20150416,Witness Defends Officer Who Rammed Suspect With Patrol Car,"['Abc News', 'April']"," -- A man who witnessed an Arizona police officer ramming a suspect with his patrol car defended the officer’s actions -- and said he worried a shootout could have ensued if the officer hadn't acted.\n\n“If they got into a gunfight, there would have been rounds flying all over the place,” said David Lightfoot, who watched the situation unfold on Feb. 19 in front of his office.\n\nVideo from the incident gained national attention after it was released Tuesday. Dash cam footage from Marana Police Officer Michael Rapiejko’s car shows him ramming his vehicle into Mario Valencia, a suspect who..."
3,https://abcnews.go.com/International/wireStory/upheaval-venezuelas-maduro-touts-anniversary-63160283,20190520,"Despite upheaval, Venezuela's Maduro touts an anniversary","['Abc News', 'Christopher Torchia Associated Press', 'May']","Venezuelan President Nicolás Maduro is celebrating the first anniversary of his disputed re-election amid a growing humanitarian crisis and political upheaval\n\nCARACAS, Venezuela -- Venezuelan President Nicolás Maduro on Monday celebrated the anniversary of his disputed re-election amid a growing humanitarian crisis and political upheaval.\n\nMaduro tweeted that the May 20, 2018, election, was a ""victory"" for Venezuelans, though the opposition and many countries have derided it as unfair.\n\nIn a speech at a government-organized rally, Maduro said he is prepared to face the opposition in..."
4,https://web.archive.org/web/20130502010216/http://abcnews.go.com:80/International/wireStory/safrica-president-finds-mandela-good-shape-19065585,20130429,Mandela Appears an Ailing Man in Video,"['Abc News', 'Authors List', 'Michelle Faul Associated Press']","South African President Jacob Zuma said he found Nelson Mandela ""in good shape and in good spirits"" Monday, but a video of his encounter with the ailing anti-apartheid icon belies those cheery words, showing him with a vacant look on his face.\n\nIt's been more than three weeks since Mandela was released after a 10-day stay in the hospital, the third time in five months that he was hospitalized for a recurring lung infection.\n\n""We sa

In [6]:
df_merge = pd.concat([df, df_others]).reset_index(drop=True)

In [7]:
df_merge.text.str.len().describe()

count    112452.000000
mean       3663.640664
std        4415.347872
min         204.000000
25%        1777.000000
50%        2923.000000
75%        4262.000000
max      100000.000000
Name: text, dtype: float64

In [8]:
df_merge = df_merge[~df_merge.text.duplicated()]
df_merge = df_merge[['title', 'date', 'text','url']]
df_merge.date = df_merge.date .apply(lambda x: re.sub('(\d\d\d\d)-?(\d\d)-?(\d\d)','\\1-\\2-\\3',str(x)))
df_merge.sort_values('date')

,title,date,text,url
0,1900's New Century Hype Was Millennial,2000-12-29 18:29:02+00:00,"Dec. 31, 2000  -- For New Year’s Eve 1900, New Yorkers pulled out all the stops to welcome the new century with fireworks, a lineup of bands and speakers, and blazing displays of electric lights. It all followed a mountain of hype in the media about the coming of the 20th century and the passing of the 19th.\n\nIt’s 100 years later. A few cities are planning big celebrations.\n\nBy and large, though, we’ve been there, done that.\n\nLike us, our 1800 and 1900 ancestors aired the debate on the start of the new century. And like us, scientists and scholars insisted upon the conclusion that t...",https://abcnews.go.com/US/1900s-century-hype-millennial/story?id=89978
1,"2 Dead, 13 Hurt in Calif. School Shooting",2001-03-05 05:00:00+00:00,"March 5, 2001 -- Smiling as he pulled the trigger, a 15-year-old freshman shot and killed two students and wounded 13 other people at a San Diego-area high school this morning, authorities and witnesses said.\n\nSheriff's deputies took the suspect into custody at Santana High School in Santee, just east of San Diego, where the deadly rampage took place around 9:20 a.m.\n\nTwo students were killed and 13 other people wounded, said San Diego County Sheriff William Kolender.\n\nA 14-year-old boy died at the scene, and a 15-year-old boy who had been shot in the head died at Grossmont Hospital....",https://abcnews.go.com/US/dead-13-hurt-calif-school-shooting/story?id=92482
2,Shot Fired At Pa. High School,2001-03-07 05:00:00+00:00," -- An eighth-grade girl is in custody afer shooting a fellow female student today during lunch at a Roman Catholic high school in Williamsport, Pa., authorities said.\n\nWilliamsport Police said an argument between the two girls sparked the shooting, which occurred at around noon at Bishop Neumann Junior-Senior High. They said the suspect, who is 14, shot the 13-year-old victim from behind.\n\nPolice said the girls knew each other and that the shooting was ""not an act of random violence."" The girls had been engaged in an ongoing feud, according to Lycoming County District Attorney TomMar...",https://abcnews.go.com/US/shot-fired-pa-high-school/story?id=93896
3,"FBI, Private Sleuths Probe Ukraine Mystery",2001-03-27 17:51:45+00:00,"March 28, 2001 -- The mystery surrounding a headless body found in the woods outside Kiev has created a dark cloud over the Ukrainian head of state.\n\nPresident Leonid Kuchma has been caught up in the storm around the body, believed to be that of opposition journalist Heorhiy Gongadze, ever since a former presidential security officer now in hiding in Europe announced he had tape recordings of the president talking about getting rid of the reporter.\n\nBut more than four months after the body was found, it has not been conclusively identified, and the integrity of the tapes has been quest...",https://abcnews.go.com/International/story?id=81346&page=1
4,Pot Smoker Demands Apology From Delta,2001-08-16 21:03:33+00:00,"Aug. 16, 2001 -- Irvin Rosenfeld needs marijuana.\n\nHe's not an addict, but the 49-year-old stockbroker has a rare and painful bone disease that 19 years ago led the federal government to not only prescribe marijuana for him, but to provide him with the narcotic as well.\n\nHe is one of only eight people in the United States legally allowed to use marijuana.\n\nIn March, Rosenfeld was invited to file a brief for a U.S. Supreme Court hearing on medical marijuana. As he said he had done many times before, he booked his flight from Fort Lauderdale, Fla., to Washington on Delta, telling the a...",https://abcnews.go.com/US/pot-smoker-demands-apology-delta/story?id=92638
...,...,...,...,...
45518,Police officer died of 'natural' causes day after Capitol riot: Medical examiner,2021-04-19 22:55:16+00:00,"Authorities haven't said if the assault directly contributed to his death.\n\nU.S. Capitol Police Officer Brian Sicknick died of ""natural"" causes on

In [9]:
df_merge.to_csv('abc_01_21.csv', index=False)